<a href="https://colab.research.google.com/github/sagihaider/IADS_SummerSchool_NN_2019/blob/master/Day1_CNN_CarsVsPlanes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from zipfile import ZipFile
file_name = "/content/drive/My Drive/CarsVsPlanes.zip"

with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [0]:
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
import skimage.io
import skimage.transform
from sklearn.preprocessing import label_binarize
from keras import layers
from keras import models
from keras import optimizers
import warnings
warnings.filterwarnings('ignore')

In [0]:
#Directory names of the folders that contain the images that need to be trained and tested for.

trainDir = 'CarsVsPlanes/train'
testDir = 'CarsVsPlanes/test'


In [0]:
#The list of class names for which label binarizer is run
class_names = ['cars', 'planes']

#channels = 3 ==> RGB or HSV images, channels = 1 ==> Greyscale images
channels = 3

#Normalization value should be 255 for RGB or Greyscale images. It should be 1 for HSV images.
normalizationVal = 255.0

#Other parameters
epochs = 100
batchsize = 32
learningRate = 0.001


In [12]:
#Converting the images to numpy arrays
X_train = []
y_train = []

X_test = []
y_test = []

#For the train images
for dirname in os.listdir(trainDir):
    if dirname in class_names:
        print(dirname)
        classdir = trainDir + '//' + dirname
        for filename in os.listdir(classdir):
            if filename.endswith('.jpg'):
                fnWithPath = classdir + '//' + filename
                image_data = skimage.io.imread(fnWithPath)
                new_image_data = skimage.transform.resize(image_data,(150,150,channels))
                new_image_data = new_image_data.reshape((1, 150, 150, channels)).astype(np.float32) / normalizationVal
                X_train.append(new_image_data)
                y_train.append(dirname)
        
#For the validation images
for dirname in os.listdir(testDir):
    if dirname in class_names:
        classdir = testDir + '//' + dirname
        for filename in os.listdir(classdir):
            if filename.endswith('.jpg'):
                fnWithPath = classdir + '//' + filename
                image_data = skimage.io.imread(fnWithPath)
                new_image_data = skimage.transform.resize(image_data,(150,150,channels))
                new_image_data = new_image_data.reshape((1, 150, 150, channels)).astype(np.float32) / normalizationVal
                X_test.append(new_image_data)
                y_test.append(dirname)
                



cars
planes


In [14]:
print(np.size(X_train))
print(np.size(y_train)) #The total number of train images
print(np.size(X_test))
print(np.size(y_test))  #The total number of test images

27000000
400
6750000
100


In [0]:
X_train = np.reshape(X_train,(np.size(y_train),150,150,channels))
y_train = np.reshape(y_train,(np.size(y_train),1))
X_test = np.reshape(X_test,(np.size(y_test),150,150,channels))
y_test = np.reshape(y_test,(np.size(y_test),1))

In [0]:
y_train = label_binarize(y_train, classes = class_names)
y_test = label_binarize(y_test, classes = class_names)

In [16]:
#%% Define model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 15, 15, 128)       147584    
__________

In [10]:
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])


#%% Executing the model
history = model.fit(X_train, 
                    y_train, 
                    shuffle         = True,
                    epochs          = epochs, 
                    verbose         = 1, 
                    batch_size      = batchsize)

W0724 12:50:55.847667 140134458267520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0724 12:50:55.853083 140134458267520 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W0724 12:50:55.858465 140134458267520 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/100
400/400 [==============================] - 4s 10ms/step - loss: 0.6934 - acc: 0.4600
Epoch 2/100
400/400 [==============================] - 0s 974us/step - loss: 0.6932 - acc: 0.4975
Epoch 3/100
400/400 [==============================] - 0s 964us/step - loss: 0.6923 - acc: 0.5325
Epoch 4/100
400/400 [==============================] - 0s 967us/step - loss: 0.6887 - acc: 0.5850
Epoch 5/100
400/400 [==============================] - 0s 972us/step - loss: 0.6737 - acc: 0.5975
Epoch 6/100
400/400 [==============================] - 0s 957us/step - loss: 0.6500 - acc: 0.6650
Epoch 7/100
400/400 [==============================] - 0s 951us/step - loss: 0.6167 - acc: 0.7025
Epoch 8/100
400/400 [==============================] - 0s 976us/step - loss: 0.5803 - acc: 0.7250
Epoch 9/100
400/400 [==============================] - 0s 950us/step - loss: 0.5668 - acc: 0.7125
Epoch 10/100
400/400 [==============================] - 0s 965us/step - loss: 0.5233 - acc: 0.7425
Epoch 11/100
400/400

In [11]:
#%% Predict Model

pred_y=model.predict(X_test)
pred_y_class=model.predict_classes(X_test)
pred_y_class_prob=model.predict_proba(X_test)
from sklearn.metrics import accuracy_score

# Print accuracy
accuracy_score(y_test, pred_y_class)

0.81